In [18]:
import emoji
import malaya
import nltk
import pandas as pd
import string
import re
import string
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from spellchecker import SpellChecker
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import logging
logging.basicConfig(level=logging.INFO)
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')

# expanding the dislpay of tweet text for columns and rows
pd.set_option('display.max_colwidth', None)
pd.set_option("display.max_rows", None)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


# Data Preprocessing

# First Stage

In [15]:
# Read dataframe from csv which containing the tweets and their details scraped from Twitter.
sunway_tweets_df = pd.read_csv('sunway_tweets.csv')
taylor_tweets_df = pd.read_csv('taylor_tweets.csv')
ucsi_tweets_df = pd.read_csv('ucsi_tweets.csv')
um_tweets_df = pd.read_csv('um_tweets.csv')
upm_tweets_df = pd.read_csv('upm_tweets.csv')
ukm_tweets_df = pd.read_csv('ukm_tweets.csv')

In [16]:
# First 5 rows of sunway_tweets_df
sunway_tweets_df.head()

,User,Date_Created,Number_of_Likes,Source_of_Tweet,Tweet,Retweeted_Tweets,Language,Location
0,jkmkhoo,2022-09-29 13:30:47+00:00,2,Twitter Web App,@Karl_A_Miller This restaurant at near Sunway University might suit a British palette well 😉 https://t.co/nfirVM3Uzl,NaN,en,"Nottingham, England"
1,eurospan,2022-09-29 09:46:32+00:00,0,Twitter Web App,Exciting news: Sunway University Press have launched a brand-new short story competition inspired by the UN Sustainable Development Goals. You can find out more on the Eurospan blog now: https://t.co/KIq59fXXHo https://t.co/8gNfzgKKwS,NaN,en,"London, UK"
2,SunwayCPH,2022-09-29 08:30:05+00:00,4,Twitter for iPhone,"Check this out! \n\nOur Chief Planetary Health Scientist, Dr. @RenzoGuinto is in the latest episode of the Katipunan Dialogue Podcast 🗣️\n\n#PlanetaryHealth #SunwayCPH #SunwayUniversity #UPCIDS #Podcast https://t.co/MiwyfSHPO8",NaN,en,Malaysia
3,keenguyencaphe,2022-09-29 04:13:02+00:00,5,Twitter for Android,Thus that was an amazing weekend as Kee Nguyễn went on the loose to bring Vietnam's Finest Coffee into Sunway University. Big Kudos to keenguyensunway for all the amazing hardwork and we hope that one sip brings you all to Vietnam or back to Vietnam already 😉\n\nGot events? DM https://t.co/nmduRwHv8m,NaN,en,Malaysia
4,Mia7Jane,2022-09-28 11:16:17+00:00,1,Twitter for Android,"@elee618 He asked :""Dr Jane from Sunway University?"", but not Dr Jane from Malaysia... 😆 #mosthappeningcampus",NaN,en,Klang


## Check values in 'Retweeted_Tweets' column for each dataframe

In [17]:
print(sunway_tweets_df.Retweeted_Tweets.value_counts())
print(taylor_tweets_df.Retweeted_Tweets.value_counts())
print(ucsi_tweets_df.Retweeted_Tweets.value_counts())
print(um_tweets_df.Retweeted_Tweets.value_counts())
print(upm_tweets_df.Retweeted_Tweets.value_counts())
print(ukm_tweets_df.Retweeted_Tweets.value_counts())

Series([], Name: Retweeted_Tweets, dtype: int64)
Series([], Name: Retweeted_Tweets, dtype: int64)
Series([], Name: Retweeted_Tweets, dtype: int64)
Series([], Name: Retweeted_Tweets, dtype: int64)
Series([], Name: Retweeted_Tweets, dtype: int64)
Series([], Name: Retweeted_Tweets, dtype: int64)


No data in *Retweeted_Tweets* column in each dataframe, which means there is no retweet. Hence the columns could be dropped.

In [18]:
# Drop 'Retweeted_Tweets' column
sunway_tweets_df = sunway_tweets_df.drop('Retweeted_Tweets', axis=1)
taylor_tweets_df = taylor_tweets_df.drop('Retweeted_Tweets', axis=1)
ucsi_tweets_df = ucsi_tweets_df.drop('Retweeted_Tweets', axis=1)
um_tweets_df = um_tweets_df.drop('Retweeted_Tweets', axis=1)
upm_tweets_df = upm_tweets_df.drop('Retweeted_Tweets', axis=1)
ukm_tweets_df = ukm_tweets_df.drop('Retweeted_Tweets', axis=1)

## Check value in 'Language' column

In [19]:
# Check the count of languages in sunway_tweets_df
sunway_tweets_df.Language.value_counts()

en     490
in      86
ja      15
tl       5
zh       4
qme      3
ar       3
ro       3
qht      3
und      1
hi       1
Name: Language, dtype: int64

In [20]:
# To see what language 'in' is
sunway_tweets_df.loc[sunway_tweets_df.Language=='in']['Tweet']

18                                                                     #repost - JAWATAN KOSONG DI SUNWAY UNIVERSITY - 21 OKTOBER 2022\n\nSila rujuk iklan jawatan &amp; baca syarat kelayakan terlebih dahulu sebelum memohon melalui pautan di bawah:\nhttps://t.co/YFVTBltOpx\n\nSUNWAY UNIVERSITY https://t.co/YFVTBltOpx
24                                                                                                  IIUM Holdings Sdn Bhd - 15 Oktober 2022 - [https://t.co/cW1bpBccPA]\n\nSri Kota Specialist Medical Centre - 7 Oktober 2022 - [https://t.co/G0Cv38KcZ0]\n\nSunway University - 21 Oktober 2022 - [https://t.co/YFVTBlumf5]
25                                                                                                                                                                                                                                              Jawatan Kosong di Sunway University - 21 Oktober 2022 https://t.co/jJRELQflYA
26                                            

Language 'in' seems is malay language.

In [21]:
sunway_tweets_df.loc[sunway_tweets_df.Language=='zh']['Tweet']

75                                                              @leilei1118 我是说我这张 Twitter profile pic la!! 其实这是五年前我在 Sunway University 拍的，拿到 bonus过后心情超好，也是我难得笑得这么开心的照片，最满意的一张，所以不舍得换的😆✌🏻
94                                                                                                      @leilei1118 其实我这个Twitter Pic 就是我在 Sunway University上班午餐的时候拍的😆😆\n心情超好，拿bonus 超开心。=P
95                                                                                           @leilei1118 你呢？！你是做会计的是吗？！还是保险业务？\n\n我是 Sunway University的前员工，现在其中一间双威的affiliate公司担任Officer✌🏻
579    拿督林金海很荣幸获双威大学邀请于2021年10月22日，在线上与众学子分享主题《Focus &amp; Transform》。当天拿督林金海将与学子们分享其创业历程，同时也将阐述如何将经验及知识运用到实际的商业环境中。\n\n#khlimgroup #sunwayuniversity #october2021 https://t.co/1luu3ZC6sn
Name: Tweet, dtype: object

Language 'zh' is Mandarin.

In [22]:
# Check the count of languages in taylor_tweets_df
taylor_tweets_df.Language.value_counts()

en     657
in      69
qht     29
ja       9
qme      7
pt       2
ro       2
de       2
zh       2
ar       1
tl       1
und      1
Name: Language, dtype: int64

In [23]:
# Check the count of languages in um_tweets_df
um_tweets_df.Language.value_counts()

en     797
in     505
qht     49
ja      18
qme     12
und      8
ro       5
zh       4
pt       4
sl       3
th       3
hi       3
et       3
ar       2
bn       2
fr       2
tl       2
lt       1
it       1
es       1
ur       1
tr       1
ca       1
Name: Language, dtype: int64

Since we only require English and Malay language for model training, the rows with languages other than those could be dropped from all dataframes.

In [24]:
# Remove the rows with languages other than 'en' and 'in'
sunway_tweets_df = sunway_tweets_df.loc[(sunway_tweets_df.Language=='en') | (sunway_tweets_df.Language=='in')]
taylor_tweets_df = taylor_tweets_df.loc[(taylor_tweets_df.Language=='en') | (taylor_tweets_df.Language=='in')]
ucsi_tweets_df = ucsi_tweets_df.loc[(ucsi_tweets_df.Language=='en') | (ucsi_tweets_df.Language=='in')]
um_tweets_df = um_tweets_df.loc[(um_tweets_df.Language=='en') | (um_tweets_df.Language=='in')]
upm_tweets_df = upm_tweets_df.loc[(upm_tweets_df.Language=='en') | (upm_tweets_df.Language=='in')]
ukm_tweets_df = ukm_tweets_df.loc[(ukm_tweets_df.Language=='en') | (ukm_tweets_df.Language=='in')]

## Check value in 'Location' column for each dataframe

In [25]:
# Check the count of location in sunway_tweets_df
sunway_tweets_df.Location.value_counts()

Malaysia                          77
Bandar Sunway, Malaysia           73
Seoul, Republic of Korea          26
Kuala Lumpur, Malaysia            22
Sunway City                       15
Selangor, Malaysia                14
Kuala Lumpur                      14
Kuala Lumpur Malaysia             13
Kuala Lumpur                      12
Petaling Jaya, Selangor           11
Perak, Malaysia                   10
Kuala Lumpur City                  7
Kuantan, Pahang                    6
Kuala Lumpur Federal Territory     6
Petaling Jaya                      4
Lagos, Nigeria                     4
Berlin, Boston, Budapest           4
Estonia, Malaysia, Philippines     4
Petaling, Selangor                 4
Nigeria                            3
Mumbai, India                      3
Singapore                          3
Australia                          3
South East Asia                    3
London, UK                         3
Subang Jaya                        3
DHAKA, BANGLADESH                  3
L

It could be observed that there are some tweets' location not from Malaysia. Let's see some tweets from location other than Malaysia.

In [26]:
# Display some tweets from location other than Malaysia.

print(sunway_tweets_df.loc[sunway_tweets_df.Location=='Berlin']['Tweet'],'\n\n')
print(sunway_tweets_df.loc[sunway_tweets_df.Location=='Nigeria']['Tweet'],'\n\n')
print(sunway_tweets_df.loc[sunway_tweets_df.Location=='Indonesia']['Tweet'],'\n')
print(sunway_tweets_df.loc[sunway_tweets_df.Location=='United Arab Emirates']['Tweet'],'\n\n')
print(sunway_tweets_df.loc[sunway_tweets_df.Location=='Nigeria']['Tweet'],'\n\n')

354    'Great Thinkers, Great Minds :  #Sunway University Professorial Lecture Series' a @SunwayU book on #ScienceOpen: https://t.co/tStisYiBW0
Name: Tweet, dtype: object 


88          Taught International Scholarships at Sunway University – Malaysia,  https://t.co/pETLc4HU6o
92     2022 Taught International Scholarships at Malaysia's Sunway University\n https://t.co/DasVarybya
155                    Sunway University International Scholarships in Malaysia https://t.co/BVmo7MShTj
Name: Tweet, dtype: object 


430    Panelists:\n• Wing Thye Woo, Research Professor, Sunway University, Malaysia\n• John McArthur, Director, Center for Sustainable Development, the Brookings Institute\n• Lynn Wagner, Senior Director, The International Institute for Sustainable Development (IISD)
Name: Tweet, dtype: object 

90    Malaysia:\n\nSunway University malaysia\n50% discount.\n\nhttps://t.co/I5omi7zjgh
Name: Tweet, dtype: object 


88          Taught International Scholarships at Sunway University – 

In [27]:
# Check the count of languages in um_tweets_df
um_tweets_df.Location.value_counts()

Malaysia                                                                        148
Kuala Lumpur                                                                    112
Universiti Malaya                                                               108
Faculty of Law, UM                                                               66
Kuala Lumpur, Malaysia                                                           51
Kuala Lumpur, Malaysia                                                           32
Kuala Lumpur Federal Territory                                                   26
University Malaya, KL                                                            26
Petaling Jaya, Selangor                                                          20
Malaya                                                                           15
Kuala Lumpur City                                                                11
Indonesia                                                                   

In [28]:
# Display some tweets from location other than Malaysia.

print(um_tweets_df.loc[um_tweets_df.Location=='Manila City']['Tweet'],'\n\n')
print(um_tweets_df.loc[um_tweets_df.Location=='Singapore']['Tweet'],'\n\n')
print(um_tweets_df.loc[um_tweets_df.Location=='Lahore, Pakistan']['Tweet'],'\n\n')

32     First to present is Professor Vincent Hoe from the Faculty of Medicine, University of Malaya, Malaysia. He will be discussing the threats and opportunities among health workers in the time of emergencies and disasters.\n\n@53rdAPACPH @UPCPH\n#53rdAPACPHConference https://t.co/PbdJrRSRAe
33    Dr. Low Wah Yun, immediate past president of the APACPH and Deputy Executive Director for Research and Internationalization from the University of Malaya in Malaysia, receives the APACPH Public Health Recognition Award.\n\n@53rdAPACPH @UPCPH\n#53rdAPACPHConference https://t.co/dd68mnIVmN
35                                                                                                        Speakers and participants wrap up the Getting Your Paper Published Workshop by the University of Malaya, Malaysia with a photo opportunity.\n\n#53rdAPACPHConference https://t.co/MFe1Hx3R5o
37         On Publishing Qualitative Research Papers\n\nProfessor Dr. Wah Yun Low, \nFaculty of Medicine, Universit

It seems like the tweets from the locations other than Malaysia in each dataframe are relevant to each university. Hence we will remain the tweets from other locations and remove the *Location* column since it has no effect to the outcome.

In [29]:
# Drop 'Location' column.
sunway_tweets_df = sunway_tweets_df.drop('Location', axis=1)
taylor_tweets_df = taylor_tweets_df.drop('Location', axis=1)
ucsi_tweets_df = ucsi_tweets_df.drop('Location', axis=1)
um_tweets_df = um_tweets_df.drop('Location', axis=1)
upm_tweets_df = upm_tweets_df.drop('Location', axis=1)
ukm_tweets_df = ukm_tweets_df.drop('Location', axis=1)

## Username (@username) Removal

In [30]:
# To display all the mentioned username and hashtags in tweets regarding Sunway University. 
sunway_tweets_df.Tweet.str.findall('@\w+')

0                                                                               [@Karl_A_Miller]
1                                                                                             []
2                                                                                 [@RenzoGuinto]
3                                                                                             []
4                                                                                     [@elee618]
5                                                       [@JemilahMahmood, @saysdotcom, @SunwayU]
6                                                                                    [@myUNICEF]
7                                                                                             []
8      [@myUNICEF, @SunwayU, @futurexfarm, @RenzoGuinto, @danial_ari, @JasminIrisha, @meisarahf]
9                                                                                             []
10                            

In [31]:
# Create a copy of each dataframe for usernames and hastags removal.
sunway_df_username_removal = sunway_tweets_df.copy()
taylor_df_username_removal = taylor_tweets_df.copy()
ucsi_df_username_removal = ucsi_tweets_df.copy()
um_df_username_removal = um_tweets_df.copy()
upm_df_username_removal = upm_tweets_df.copy()
ukm_df_username_removal = ukm_tweets_df.copy()

# To remove the mentioned username and hashtags in all dataframe
sunway_df_username_removal.Tweet = sunway_tweets_df.Tweet.str.replace('@\w+', '', regex=True)
taylor_df_username_removal.Tweet = taylor_tweets_df.Tweet.str.replace('@\w+', '', regex=True)
ucsi_df_username_removal.Tweet = ucsi_tweets_df.Tweet.str.replace('@\w+', '', regex=True)
um_df_username_removal.Tweet = um_tweets_df.Tweet.str.replace('@\w+', '', regex=True)
upm_df_username_removal.Tweet = upm_tweets_df.Tweet.str.replace('@\w+', '', regex=True)
ukm_df_username_removal.Tweet = ukm_tweets_df.Tweet.str.replace('@\w+', '', regex=True)

## Hyperlinks removal

In [32]:
# To display hyperlinks in tweets regarding Sunway University.
sunway_df_username_removal.Tweet.str.findall(r'http\S+')

0                                                                                     [https://t.co/nfirVM3Uzl]
1                                                            [https://t.co/KIq59fXXHo, https://t.co/8gNfzgKKwS]
2                                                                                     [https://t.co/MiwyfSHPO8]
3                                                                                     [https://t.co/nmduRwHv8m]
4                                                                                                            []
5                                                                                     [https://t.co/BnpDXxvKNS]
6                                                                                     [https://t.co/CS9tGM64GV]
7                                                                                     [https://t.co/was3qvsIzY]
8                                                                                     [https://t.co/kEn8

In [33]:
# Create a copy of each dataframe for hyperlinks removal.
sunway_df_link_removal = sunway_df_username_removal.copy()
taylor_df_link_removal = taylor_df_username_removal.copy()
ucsi_df_link_removal = ucsi_df_username_removal.copy()
um_df_link_removal = um_df_username_removal.copy()
upm_df_link_removal = upm_df_username_removal.copy()
ukm_df_link_removal = ukm_df_username_removal.copy()

# To remove hyperlinks.
sunway_df_link_removal.Tweet = sunway_df_username_removal.Tweet.str.replace(r'http\S+', '', regex=True)
taylor_df_link_removal.Tweet = taylor_df_username_removal.Tweet.str.replace(r'http\S+', '', regex=True)
ucsi_df_link_removal.Tweet = ucsi_df_username_removal.Tweet.str.replace(r'http\S+', '', regex=True)
um_df_link_removal.Tweet = um_df_username_removal.Tweet.str.replace(r'http\S+', '', regex=True)
upm_df_link_removal.Tweet = upm_df_username_removal.Tweet.str.replace(r'http\S+', '', regex=True)
ukm_df_link_removal.Tweet = ukm_df_username_removal.Tweet.str.replace(r'http\S+', '', regex=True)

In [34]:
# To make sure all the hyperlinks have been removed in tweets regarding Sunway University.
sunway_df_link_removal.Tweet.str.findall('http')

0      []
1      []
2      []
3      []
4      []
5      []
6      []
7      []
8      []
9      []
10     []
11     []
13     []
14     []
15     []
16     []
17     []
18     []
19     []
20     []
21     []
22     []
23     []
24     []
25     []
26     []
27     []
28     []
29     []
30     []
31     []
32     []
33     []
34     []
35     []
36     []
37     []
38     []
39     []
40     []
41     []
42     []
43     []
44     []
45     []
46     []
47     []
48     []
49     []
50     []
51     []
52     []
53     []
54     []
55     []
56     []
57     []
58     []
59     []
60     []
61     []
62     []
63     []
64     []
65     []
66     []
67     []
68     []
69     []
70     []
71     []
72     []
73     []
74     []
76     []
77     []
78     []
79     []
80     []
81     []
82     []
83     []
84     []
85     []
86     []
87     []
88     []
89     []
90     []
91     []
92     []
93     []
96     []
97     []
98     []
99     []
102    []
104    []
106    []
107    []


No more hyperlinks in all the tweets regarding Sunway University.

## 	Punctuation removal, with the exception of the comma, apostrophe, full stop, exclamation mark, and question mark

In [35]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [36]:
def remove_punctuation(text):
    text = ''.join(char for char in text if char not in '"#“”$%&\'()*+-/:;<=>‘@[\\]^_`’🥹{|}•~')
    text = re.sub('\s\s+','', text)
    return text

In [37]:
# Create a copy of each dataframe for punctuation removal.
sunway_df_punct_removal = sunway_df_link_removal.copy()
taylor_df_punct_removal = taylor_df_link_removal.copy()
ucsi_df_punct_removal = ucsi_df_link_removal.copy()
um_df_punct_removal = um_df_link_removal.copy()
upm_df_punct_removal = upm_df_link_removal.copy()
ukm_df_punct_removal = ukm_df_link_removal.copy()

# To remove punctuation.
sunway_df_punct_removal.Tweet = sunway_df_link_removal.Tweet.apply(lambda x: remove_punctuation(x))
taylor_df_punct_removal.Tweet = taylor_df_link_removal.Tweet.apply(lambda x: remove_punctuation(x))
ucsi_df_punct_removal.Tweet = ucsi_df_link_removal.Tweet.apply(lambda x: remove_punctuation(x))
um_df_punct_removal.Tweet = um_df_link_removal.Tweet.apply(lambda x: remove_punctuation(x))
upm_df_punct_removal.Tweet = upm_df_link_removal.Tweet.apply(lambda x: remove_punctuation(x))
ukm_df_punct_removal.Tweet = ukm_df_link_removal.Tweet.apply(lambda x: remove_punctuation(x))

In [38]:
sunway_df_punct_removal.Tweet

0                                                                                                                                                                                                                     This restaurant at near Sunway University might suit a British palette well 😉
1                                                                                                           Exciting news Sunway University Press have launched a brandnew short story competition inspired by the UN Sustainable Development Goals. You can find out more on the Eurospan blog now
2                                                                                                                   Check this out!Our Chief Planetary Health Scientist, Dr.is in the latest episode of the Katipunan Dialogue Podcast 🗣️PlanetaryHealth SunwayCPH SunwayUniversity UPCIDS Podcast 
3                 Thus that was an amazing weekend as Kee Nguyễn went on the loose to bring Vietnams Finest Coffee into Sunw

All punctuations are removed except comma, full stop, exclamation mark, and question mark.

## Extra whitespaces removal

In [39]:
# To display extra whitespaces in tweets regarding Sunway University.
sunway_df_punct_removal.Tweet.str.findall('\s\s+')

0      []
1      []
2      []
3      []
4      []
5      []
6      []
7      []
8      []
9      []
10     []
11     []
13     []
14     []
15     []
16     []
17     []
18     []
19     []
20     []
21     []
22     []
23     []
24     []
25     []
26     []
27     []
28     []
29     []
30     []
31     []
32     []
33     []
34     []
35     []
36     []
37     []
38     []
39     []
40     []
41     []
42     []
43     []
44     []
45     []
46     []
47     []
48     []
49     []
50     []
51     []
52     []
53     []
54     []
55     []
56     []
57     []
58     []
59     []
60     []
61     []
62     []
63     []
64     []
65     []
66     []
67     []
68     []
69     []
70     []
71     []
72     []
73     []
74     []
76     []
77     []
78     []
79     []
80     []
81     []
82     []
83     []
84     []
85     []
86     []
87     []
88     []
89     []
90     []
91     []
92     []
93     []
96     []
97     []
98     []
99     []
102    []
104    []
106    []
107    []


In [40]:
# Create a copy of each dataframe for extra whitespaces removal.
sunway_df_exw_removal = sunway_df_punct_removal.copy()
taylor_df_exw_removal = taylor_df_punct_removal.copy()
ucsi_df_exw_removal = ucsi_df_punct_removal.copy()
um_df_exw_removal = um_df_punct_removal.copy()
upm_df_exw_removal = upm_df_punct_removal.copy()
ukm_df_exw_removal = ukm_df_punct_removal.copy()

# To remove extra whitespaces.
sunway_df_exw_removal.Tweet = sunway_df_punct_removal.Tweet.str.replace('\s\s+', '', regex=True)
taylor_df_exw_removal.Tweet = taylor_df_punct_removal.Tweet.str.replace('\s\s+', '', regex=True)
ucsi_df_exw_removal.Tweet = ucsi_df_punct_removal.Tweet.str.replace('\s\s+', '', regex=True)
um_df_exw_removal.Tweet = um_df_punct_removal.Tweet.str.replace('\s\s+', '', regex=True)
upm_df_exw_removal.Tweet = upm_df_punct_removal.Tweet.str.replace('\s\s+', '', regex=True)
ukm_df_exw_removal.Tweet = ukm_df_punct_removal.Tweet.str.replace('\s\s+', '', regex=True)

In [41]:
# Compare the tweets before and after extra whitespaces removal.
# Before:
sunway_df_link_removal.Tweet.iloc[2]

'Check this out! \n\nOur Chief Planetary Health Scientist, Dr.  is in the latest episode of the Katipunan Dialogue Podcast 🗣️\n\n#PlanetaryHealth #SunwayCPH #SunwayUniversity #UPCIDS #Podcast '

In [42]:
# After:
sunway_df_exw_removal.Tweet.iloc[2]

'Check this out!Our Chief Planetary Health Scientist, Dr.is in the latest episode of the Katipunan Dialogue Podcast 🗣️PlanetaryHealth SunwayCPH SunwayUniversity UPCIDS Podcast '

In [43]:
# To check there is '\n', which is not removed.
sunway_df_exw_removal.Tweet.str.findall('\n')

0                                    []
1                                    []
2                                    []
3                                    []
4                                    []
5                                    []
6                                    []
7                                    []
8                                    []
9                                    []
10                                   []
11                                   []
13                                   []
14                                   []
15                                   []
16                                   []
17                                   []
18                                   []
19                                   []
20                                   []
21                                   []
22                                   []
23                                   []
24                                   []
25                                   []


It seems like there are quite a number of '\n' which have not been removed.

In [44]:
# To remove '\n'.
sunway_df_exw_removal.Tweet = sunway_df_exw_removal.Tweet.str.replace('\n', '', regex=True)
taylor_df_exw_removal.Tweet = taylor_df_exw_removal.Tweet.str.replace('\n', '', regex=True)
ucsi_df_exw_removal.Tweet = ucsi_df_exw_removal.Tweet.str.replace('\n', '', regex=True)
um_df_exw_removal.Tweet = um_df_exw_removal.Tweet.str.replace('\n', '', regex=True)
upm_df_exw_removal.Tweet = upm_df_exw_removal.Tweet.str.replace('\n', '', regex=True)
ukm_df_exw_removal.Tweet = ukm_df_exw_removal.Tweet.str.replace('\n', '', regex=True)

## Superscript processing

In [45]:
# To check dataframe one by one whether there is superscript in tweets
# sunway_df_exw_removal.reset_index().Tweet.str.findall('[\d\w]+[²³¹⁰ⁱ⁴⁵⁶⁷⁸⁹⁺⁻⁼⁽⁾ⁿ]+')
# taylor_df_exw_removal.reset_index().Tweet.str.findall('[\d\w]+[²³¹⁰ⁱ⁴⁵⁶⁷⁸⁹⁺⁻⁼⁽⁾ⁿ]+')
# ucsi_df_exw_removal.reset_index().Tweet.str.findall('[\d\w]+[²³¹⁰ⁱ⁴⁵⁶⁷⁸⁹⁺⁻⁼⁽⁾ⁿ]+')
um_df_exw_removal.reset_index().Tweet.str.findall('[\d\w]+[²³¹⁰ⁱ⁴⁵⁶⁷⁸⁹⁺⁻⁼⁽⁾ⁿ]+')
# upm_df_exw_removal.reset_index().Tweet.str.findall('[\d\w]+[²³¹⁰ⁱ⁴⁵⁶⁷⁸⁹⁺⁻⁼⁽⁾ⁿ]+')
#ukm_df_exw_removal.reset_index().Tweet.str.findall('[\d\w]+[²³¹⁰ⁱ⁴⁵⁶⁷⁸⁹⁺⁻⁼⁽⁾ⁿ]+')

0                  []
1                  []
2                  []
3                  []
4                  []
5                  []
6                  []
7                  []
8                  []
9                  []
10                 []
11                 []
12                 []
13                 []
14                 []
15                 []
16                 []
17                 []
18                 []
19                 []
20                 []
21                 []
22                 []
23                 []
24                 []
25                 []
26                 []
27                 []
28                 []
29                 []
30                 []
31                 []
32                 []
33                 []
34                 []
35                 []
36                 []
37                 []
38                 []
39                 []
40                 []
41                 []
42                 []
43                 []
44                 []
45        

As a result, there are matches in tweets in **Taylor's University** and **UM**.

We cannot remove them since they are with significant meaning, such as Undang² means Undang-undang. Therefore we will modify them individually.

In tweets of taylor_df_exw_removal, there is only 1 tweet containing superscript, which row 141.

In [46]:
taylor_df_exw_removal.Tweet.iloc[141]

'Dulu intern dkt hotel in KL ada staff cakap students taylors university takperlu allowance sebab semua anak orang kaya...so tiap kali kena suruh OT ada events ke apa i reject mentah²Sorry mama i dah tunggu dkt KLCC nak shoppingSorry i ada party to attendLol '

In [47]:
# Change mentah² to mentah2
taylor_df_exw_removal.Tweet.iloc[141] = taylor_df_exw_removal.Tweet.iloc[141].replace('mentah²', 'mentah2')

In [48]:
# Check the tweets after superscript processing
taylor_df_exw_removal.Tweet.iloc[141]

'Dulu intern dkt hotel in KL ada staff cakap students taylors university takperlu allowance sebab semua anak orang kaya...so tiap kali kena suruh OT ada events ke apa i reject mentah2Sorry mama i dah tunggu dkt KLCC nak shoppingSorry i ada party to attendLol '

In tweets of um_df_exw_removal, there are 2 tweets containing superscript, which are row 586 and 851)

In [49]:
um_df_exw_removal.Tweet.iloc[[589,858]]

622           Pelajar Syariah amp Undang² kena ada buku ni. Dikarang oleh Prof. Madya Dr Siti Zubaidah Ismail, pensyarah di Akademi Pengajian Islam Universiti Malaya merangkap SV saya. Beliau telah menerima Anugerah Cemerlang UM 2018 kategori Penerbitan Buku. 
902    Penyelia Universiti Malaya scam penuntut PhD.Curi kerja student,memalukan student dan tiba² penuntut dibuang dari UM? Lets spread the word and make this viral ! Dah terlalu ramai pelajar yang diperlakukan sebegini tanpa apa² pembelaan.NewsBreak Trending
Name: Tweet, dtype: object

In [50]:
# Change Undang² to Undang2
um_df_exw_removal.Tweet.iloc[589] = um_df_exw_removal.Tweet.iloc[589].replace('Undang²', 'Undang2')

# Change tiba² to tiba2
um_df_exw_removal.Tweet.iloc[858] = um_df_exw_removal.Tweet.iloc[858].replace('tiba²', 'tiba2')

#Change apa² to apa2
um_df_exw_removal.Tweet.iloc[858] = um_df_exw_removal.Tweet.iloc[858].replace('apa²', 'apa2')

In [51]:
# Check the tweets after superscript processing
um_df_exw_removal.Tweet.iloc[[589,858]]

622           Pelajar Syariah amp Undang2 kena ada buku ni. Dikarang oleh Prof. Madya Dr Siti Zubaidah Ismail, pensyarah di Akademi Pengajian Islam Universiti Malaya merangkap SV saya. Beliau telah menerima Anugerah Cemerlang UM 2018 kategori Penerbitan Buku. 
902    Penyelia Universiti Malaya scam penuntut PhD.Curi kerja student,memalukan student dan tiba2 penuntut dibuang dari UM? Lets spread the word and make this viral ! Dah terlalu ramai pelajar yang diperlakukan sebegini tanpa apa2 pembelaan.NewsBreak Trending
Name: Tweet, dtype: object

There are no more tweets with superscript. 

## Removal of Non-Alphabetic Characters

We can notice that there are Chinese, Japanese, and Jawi languages in tweets in dataframes, respectively. We would then remove them from the tweets.

In [52]:
# Chinese in tweets of Taylor's University.
taylor_df_exw_removal.Tweet.iloc[386]

'trip life malaysia happy phd kl klcc 双子塔Taylors University '

In [53]:
# After removing chinese characters.
taylor_df_exw_removal.Tweet.iloc[386] = taylor_df_exw_removal.Tweet.iloc[386].replace('双子塔', '')
taylor_df_exw_removal.Tweet.iloc[386]

'trip life malaysia happy phd kl klcc Taylors University '

In [54]:
# Chinese in tweets of Sunway University.
sunway_df_exw_removal.Tweet.iloc[178]

'Our fun and modern facilities are perfect for your next business gathering.If youre looking for a meeting room in sripetaling大城堡 or sunway , be sure to book with us! ✒️sandboxspace coworking community sharedofficespace meetingroom meetingspace sunwayuniversity '

In [55]:
# After removing chinese characters.
sunway_df_exw_removal.Tweet.iloc[178] = sunway_df_exw_removal.Tweet.iloc[178].replace('大城堡', '')
sunway_df_exw_removal.Tweet.iloc[178]

'Our fun and modern facilities are perfect for your next business gathering.If youre looking for a meeting room in sripetaling or sunway , be sure to book with us! ✒️sandboxspace coworking community sharedofficespace meetingroom meetingspace sunwayuniversity '

In [56]:
# Chinese in tweets of Sunway University.
sunway_df_exw_removal.Tweet.iloc[491]

'Our fun and modern facilities are perfect for your next business gathering.If youre looking for a meeting room in sripetaling大城堡 or sunway , be sure to book with us! ✒️sandboxspace coworking community sharedofficespace meetingroom meetingspace sunwayuniversity '

In [57]:
# After removing chinese characters.
sunway_df_exw_removal.Tweet.iloc[491] = sunway_df_exw_removal.Tweet.iloc[491].replace('大城堡', '')
sunway_df_exw_removal.Tweet.iloc[491]

'Our fun and modern facilities are perfect for your next business gathering.If youre looking for a meeting room in sripetaling or sunway , be sure to book with us! ✒️sandboxspace coworking community sharedofficespace meetingroom meetingspace sunwayuniversity '

In [58]:
# Japanese in tweets of Sunway University.
sunway_df_exw_removal.Tweet.iloc[419]

'The science stream one is from sunway university while the art one is from msu ofc Im going to hear the Sunway one ¯ツ¯'

In [59]:
# After removing Japanese characters.
sunway_df_exw_removal.Tweet.iloc[419] = sunway_df_exw_removal.Tweet.iloc[419].replace('ツ', '')
sunway_df_exw_removal.Tweet.iloc[419]

'The science stream one is from sunway university while the art one is from msu ofc Im going to hear the Sunway one ¯¯'

In [60]:
# Jawi in tweets of UM.
um_df_exw_removal.Tweet.iloc[308]

'🏛 ตัวอย่างมหาวิทยาลัยในงานNational University of SingaporeUniversity of MalayaThe Chinese University of Hong KongUniversity of NottinghamNingboTongji UniversitySingapore Management University และอีกเพียบ!'

In [61]:
# After removing Jawi characters.
um_df_exw_removal.Tweet.iloc[308] = um_df_exw_removal.Tweet.iloc[308].replace('ตัวอย่างมหาวิทยาลัยในงาน', '')
um_df_exw_removal.Tweet.iloc[308] = um_df_exw_removal.Tweet.iloc[308].replace('และอีกเพียบ', '')
um_df_exw_removal.Tweet.iloc[308]

'🏛 National University of SingaporeUniversity of MalayaThe Chinese University of Hong KongUniversity of NottinghamNingboTongji UniversitySingapore Management University !'

In [62]:
# Jawi in tweets of UPM.
upm_df_exw_removal.Tweet.iloc[59]

'بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيمTerima kasih kepada seluruh kesatriasemua yang bertungkuslumus memberi kerjasama yang baik sepanjang berlangsungnya kejohanan Liga M5 ALigue amp Liga Sosial Kelab Bolasepak Universiti Putra Malaysia yang lalu. '

In [63]:
# After removing Jawi characters.
upm_df_exw_removal.Tweet.iloc[59] = upm_df_exw_removal.Tweet.iloc[59].replace('بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيم', '')
upm_df_exw_removal.Tweet.iloc[59]

'Terima kasih kepada seluruh kesatriasemua yang bertungkuslumus memberi kerjasama yang baik sepanjang berlangsungnya kejohanan Liga M5 ALigue amp Liga Sosial Kelab Bolasepak Universiti Putra Malaysia yang lalu. '

In [64]:
# Jawi in tweets of UPM.
upm_df_exw_removal.Tweet.iloc[60]

' SYABAS DAN TAHNIAH ATAS PENCAPAIAN KEPUTUSAN PEPERIKSAAN AKHIR 202120229 Sep 202212 Safar 1444 Hبِسۡـــــــــمِ ٱللهِ ٱلرَّحۡـمَـٰنِ ٱلرَّحِـــــــيمPMIUPM ingin mengucapkan Syabas dan Tahniah kepada seluruh Mahasiswa Mahasiswi Universiti Putra Malaysia atas '

In [65]:
# After removing Jawi characters.
upm_df_exw_removal.Tweet.iloc[60] = upm_df_exw_removal.Tweet.iloc[60].replace('بِسۡـــــــــمِ ٱللهِ ٱلرَّحۡـمَـٰنِ ٱلرَّحِـــــــيم', '')
upm_df_exw_removal.Tweet.iloc[60]

' SYABAS DAN TAHNIAH ATAS PENCAPAIAN KEPUTUSAN PEPERIKSAAN AKHIR 202120229 Sep 202212 Safar 1444 HPMIUPM ingin mengucapkan Syabas dan Tahniah kepada seluruh Mahasiswa Mahasiswi Universiti Putra Malaysia atas '

In [66]:
# Jawi in tweets of UPM.
upm_df_exw_removal.Tweet.iloc[194]

'بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيمBergambar bersama pihak urusetia kejohanan Liga Sosial Kelab Bolasepak Universiti Putra Malaysia dan juga pasukan yang memenangi kejohanan edisi kali pertama ini.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [67]:
# After removing Jawi characters.
upm_df_exw_removal.Tweet.iloc[194] = upm_df_exw_removal.Tweet.iloc[194].replace('بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيم', '')
upm_df_exw_removal.Tweet.iloc[194]

'Bergambar bersama pihak urusetia kejohanan Liga Sosial Kelab Bolasepak Universiti Putra Malaysia dan juga pasukan yang memenangi kejohanan edisi kali pertama ini.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [68]:
# Jawi in tweets of UPM.
upm_df_exw_removal.Tweet.iloc[195]

'بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيمMajlis penyampaian hadiah bagi kejohanan Liga Sosial Kelab Bolasepak Universiti Putra Malaysia dimana Arslan Football Club dinobatkan selaku Naib Juara.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [69]:
# After removing Jawi characters.
upm_df_exw_removal.Tweet.iloc[195] = upm_df_exw_removal.Tweet.iloc[195].replace('بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيم', '')
upm_df_exw_removal.Tweet.iloc[195]

'Majlis penyampaian hadiah bagi kejohanan Liga Sosial Kelab Bolasepak Universiti Putra Malaysia dimana Arslan Football Club dinobatkan selaku Naib Juara.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [70]:
# Jawi in tweets of UPM.
upm_df_exw_removal.Tweet.iloc[205]

'بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيمKeputusan penuh perlawanan ketujuh Liga Sosial Kelab Bolasepak Universiti Putra Malaysia diantara Arslan Football Club vs XPS FC.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [71]:
# After removing Jawi characters.
upm_df_exw_removal.Tweet.iloc[205] = upm_df_exw_removal.Tweet.iloc[205].replace('بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيم', '')
upm_df_exw_removal.Tweet.iloc[205]

'Keputusan penuh perlawanan ketujuh Liga Sosial Kelab Bolasepak Universiti Putra Malaysia diantara Arslan Football Club vs XPS FC.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [72]:
# Jawi in tweets of UPM.
upm_df_exw_removal.Tweet.iloc[209]

'بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيمPerlawanan terakhir pada hari ini Liga Sosial Kelab Bolasepak Universiti Putra Malaysia bagi memperkukuhkan kedudukan liga.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [73]:
# After removing Jawi characters.
upm_df_exw_removal.Tweet.iloc[209] = upm_df_exw_removal.Tweet.iloc[209].replace('بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيم', '')
upm_df_exw_removal.Tweet.iloc[209]

'Perlawanan terakhir pada hari ini Liga Sosial Kelab Bolasepak Universiti Putra Malaysia bagi memperkukuhkan kedudukan liga.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [74]:
# Jawi in tweets of UPM.
upm_df_exw_removal.Tweet.iloc[212]

'بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيمPerlawanan terakhir pada minggu ini Liga Sosial Kelab Bolasepak Universiti Putra Malaysia bagi memperkukuhkan kedudukan liga.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [75]:
# After removing Jawi characters.
upm_df_exw_removal.Tweet.iloc[212] = upm_df_exw_removal.Tweet.iloc[212].replace('بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيم', '')
upm_df_exw_removal.Tweet.iloc[212]

'Perlawanan terakhir pada minggu ini Liga Sosial Kelab Bolasepak Universiti Putra Malaysia bagi memperkukuhkan kedudukan liga.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [76]:
# Jawi in tweets of UPM.
upm_df_exw_removal.Tweet.iloc[224]

'بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيمCarta kedudukan Liga Sosial Kelab Bolasepak Universiti Putra Malaysia dimana Arslan Football Club menduduki tangga kedua liga pada minggu keenam perlawanan.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [77]:
# After removing Jawi characters.
upm_df_exw_removal.Tweet.iloc[224] = upm_df_exw_removal.Tweet.iloc[224].replace('بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيم', '')
upm_df_exw_removal.Tweet.iloc[224]

'Carta kedudukan Liga Sosial Kelab Bolasepak Universiti Putra Malaysia dimana Arslan Football Club menduduki tangga kedua liga pada minggu keenam perlawanan.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [78]:
# Jawi in tweets of UPM.
upm_df_exw_removal.Tweet.iloc[229]

'بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيمPerlawanan tumpuan pada hari ini Liga Sosial Kelab Bolasepak Universiti Putra Malaysia bagi memperkukuhkan kedudukan liga.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [79]:
# After removing Jawi characters.
upm_df_exw_removal.Tweet.iloc[229] = upm_df_exw_removal.Tweet.iloc[229].replace('بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيم', '')
upm_df_exw_removal.Tweet.iloc[229]

'Perlawanan tumpuan pada hari ini Liga Sosial Kelab Bolasepak Universiti Putra Malaysia bagi memperkukuhkan kedudukan liga.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [80]:
# Jawi in tweets of UPM.
upm_df_exw_removal.Tweet.iloc[231]

'بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيمTumpuan perlawanan keenam Liga Sosial Kelab Bolasepak Universiti Putra Malaysia minggu ini, InsyaAllah kita akan buat yang terbaik untuk Arslan Football Club.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [81]:
# After removing Jawi characters.
upm_df_exw_removal.Tweet.iloc[231] = upm_df_exw_removal.Tweet.iloc[231].replace('بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيم','')
upm_df_exw_removal.Tweet.iloc[231]

'Tumpuan perlawanan keenam Liga Sosial Kelab Bolasepak Universiti Putra Malaysia minggu ini, InsyaAllah kita akan buat yang terbaik untuk Arslan Football Club.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [82]:
# Jawi in tweets of UPM.
upm_df_exw_removal.Tweet.iloc[236]

'بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيمCarta kedudukan Liga Sosial Kelab Bolasepak Universiti Putra Malaysia dimana Arslan Football Club menduduki tangga kedua liga pada minggu kelima perlawanan.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [83]:
# After removing Jawi characters.
upm_df_exw_removal.Tweet.iloc[236] = upm_df_exw_removal.Tweet.iloc[236].replace('بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيم','')
upm_df_exw_removal.Tweet.iloc[236]

'Carta kedudukan Liga Sosial Kelab Bolasepak Universiti Putra Malaysia dimana Arslan Football Club menduduki tangga kedua liga pada minggu kelima perlawanan.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [84]:
# Jawi in tweets of UPM.
upm_df_exw_removal.Tweet.iloc[245]

'بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيمPerlawanan tumpuan pada hari ini Liga Sosial Kelab Bolasepak Universiti Putra Malaysia bagi memperkukuhkan kedudukan liga.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [85]:
# After removing Jawi characters.
upm_df_exw_removal.Tweet.iloc[245] = upm_df_exw_removal.Tweet.iloc[245].replace('بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيم', '')
upm_df_exw_removal.Tweet.iloc[245]

'Perlawanan tumpuan pada hari ini Liga Sosial Kelab Bolasepak Universiti Putra Malaysia bagi memperkukuhkan kedudukan liga.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [86]:
# Jawi in tweets of UPM.
upm_df_exw_removal.Tweet.iloc[249]

'بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيمTumpuan perlawanan terpenting dalam minggu ini, perlawanan kelima Liga Sosial KBS UPM turun menentang Kelab Bolasepak Universiti Putra Malaysia.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

In [87]:
# After removing Jawi characters.
upm_df_exw_removal.Tweet.iloc[249] = upm_df_exw_removal.Tweet.iloc[249].replace('بِسْــــــــــــــــــمِ اﷲِالرَّحْمَنِ اارَّحِيم', '')
upm_df_exw_removal.Tweet.iloc[249]

'Tumpuan perlawanan terpenting dalam minggu ini, perlawanan kelima Liga Sosial KBS UPM turun menentang Kelab Bolasepak Universiti Putra Malaysia.KehebatanKitaKebanggaanSemuaMenciptaSejarahMemburuKegemilangan '

## Drop Duplicated Tweets

In [88]:
# print the number of rows after removing rows with languages other than 'en' and 'in'
print(sunway_df_exw_removal.shape[0] + taylor_df_exw_removal.shape[0] + ucsi_df_exw_removal.shape[0]
     + um_df_exw_removal.shape[0] + upm_df_exw_removal.shape[0] + ukm_df_exw_removal.shape[0], 'rows')

5253 rows


In [89]:
sunway_df_drop_duplicate = sunway_df_exw_removal.drop_duplicates(subset=['Tweet'])
taylor_df_drop_duplicate = taylor_df_exw_removal.drop_duplicates(subset=['Tweet'])
ucsi_df_drop_duplicate = ucsi_df_exw_removal.drop_duplicates(subset=['Tweet'])
um_df_drop_duplicate = um_df_exw_removal.drop_duplicates(subset=['Tweet'])
upm_df_drop_duplicate = upm_df_exw_removal.drop_duplicates(subset=['Tweet'])
ukm_df_drop_duplicate = ukm_df_exw_removal.drop_duplicates(subset=['Tweet'])

In [90]:
# print the number of rows after removing duplicate tweets
print(sunway_df_drop_duplicate.shape[0] + taylor_df_drop_duplicate.shape[0] + ucsi_df_drop_duplicate.shape[0]
     + um_df_drop_duplicate.shape[0] + upm_df_drop_duplicate.shape[0] + ukm_df_drop_duplicate.shape[0], 'rows')

4157 rows


## End of stage 1

Since it is the end of first stage, we assign the name 'stage_1' to all dataframes.

In [91]:
sunway_df_stage1 = sunway_df_drop_duplicate.copy()
taylor_df_stage1 = taylor_df_drop_duplicate.copy()
ucsi_df_stage1 = ucsi_df_drop_duplicate.copy()
um_df_stage1 = um_df_drop_duplicate.copy()
upm_df_stage1 = upm_df_drop_duplicate.copy()
ukm_df_stage1 = ukm_df_drop_duplicate.copy()

# Second Stage

Split all dataframes into language 'in' and 'en' since we are going to pre-preprocess the data with malay language.

In [93]:
sunway_df_stage1_en = sunway_df_stage1.loc[sunway_df_stage1.Language == 'en'].reset_index(drop=True)
sunway_df_stage1_mal = sunway_df_stage1.loc[sunway_df_stage1.Language == 'in'].reset_index(drop=True)
taylor_df_stage1_en = taylor_df_stage1.loc[taylor_df_stage1.Language == 'en'].reset_index(drop=True)
taylor_df_stage1_mal = taylor_df_stage1.loc[taylor_df_stage1.Language == 'in'].reset_index(drop=True)
ucsi_df_stage1_en = ucsi_df_stage1.loc[ucsi_df_stage1.Language == 'en'].reset_index(drop=True)
ucsi_df_stage1_mal = ucsi_df_stage1.loc[ucsi_df_stage1.Language == 'in'].reset_index(drop=True)
um_df_stage1_en = um_df_stage1.loc[um_df_stage1.Language == 'en'].reset_index(drop=True)
um_df_stage1_mal = um_df_stage1.loc[um_df_stage1.Language == 'in'].reset_index(drop=True)
upm_df_stage1_en = upm_df_stage1.loc[upm_df_stage1.Language == 'en'].reset_index(drop=True)
upm_df_stage1_mal = upm_df_stage1.loc[upm_df_stage1.Language == 'in'].reset_index(drop=True)
ukm_df_stage1_en = ukm_df_stage1.loc[ukm_df_stage1.Language == 'en'].reset_index(drop=True)
ukm_df_stage1_mal = ukm_df_stage1.loc[ukm_df_stage1.Language == 'in'].reset_index(drop=True)

## Normalization

In [80]:
# Build a function for normalization.
def normalizefunc(i):
    normalizer = malaya.normalize.normalizer()
    norm_s = normalizer.normalize(i, normalize_year=False, normalize_telephone=False,normalize_date=False, 
                                  normalize_time=False, normalize_hingga=False,normalize_pada_hari_bulan = False, 
                                  normalize_fraction = False, normalize_ic=False, normalize_money=False, 
                                  normalize_units=False, normalize_percent=False, normalize_number=False, 
                                  normalize_entity=False)
    return norm_s

In [81]:
%%time
# Create a copy of each dataframe for normalization.
sunway_df_mal_normalize = sunway_df_stage1_mal.copy()
taylor_df_mal_normalize = taylor_df_stage1_mal.copy()
ucsi_df_mal_normalize = ucsi_df_stage1_mal.copy()
um_df_mal_normalize = um_df_stage1_mal.copy()
upm_df_mal_normalize = upm_df_stage1_mal.copy()
ukm_df_mal_normalize = ukm_df_stage1_mal.copy()


# For normalization.
sunway_df_mal_normalize.Tweet = sunway_df_stage1_mal.Tweet.apply(lambda x : normalizefunc(x))
taylor_df_mal_normalize.Tweet = taylor_df_stage1_mal.Tweet.apply(lambda x : normalizefunc(x))
ucsi_df_mal_normalize.Tweet = ucsi_df_stage1_mal.Tweet.apply(lambda x : normalizefunc(x))
um_df_mal_normalize.Tweet = um_df_stage1_mal.Tweet.apply(lambda x : normalizefunc(x))
upm_df_mal_normalize.Tweet = upm_df_stage1_mal.Tweet.apply(lambda x : normalizefunc(x))
ukm_df_mal_normalize.Tweet = ukm_df_stage1_mal.Tweet.apply(lambda x : normalizefunc(x))

INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.

INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.

INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.

INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.

INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.

...

INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.

INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.

INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.

INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.normalize:caching malaya.preprocessing.demoji inside normalizer
INFO:malaya.

CPU times: total: 1min 24s
Wall time: 2min 28s


In [82]:
sunway_df_mal_normalize.Tweet.iloc[0]

{'normalize': 'repostJAWATAN KOSONG DI SUNWAY UNIVERSITY21 OKTOBER 2022Silah rujuk iklan jawatan amp baca syarat kelayakan terlebih dahulu sebelum memohon melalui pautan di bawahSUNWAY UNIVERSITY',
 'date': {},
 'money': {}}

We can see the tweets are not in strings after normalization. We then extract the strings from dictionary under 'normalize'.

In [83]:
# To get the normalized strings.
sunway_df_mal_normalize.Tweet = sunway_df_mal_normalize.Tweet.apply(lambda x : x['normalize'])
taylor_df_mal_normalize.Tweet = taylor_df_mal_normalize.Tweet.apply(lambda x : x['normalize'])
ucsi_df_mal_normalize.Tweet = ucsi_df_mal_normalize.Tweet.apply(lambda x : x['normalize'])
um_df_mal_normalize.Tweet = um_df_mal_normalize.Tweet.apply(lambda x : x['normalize'])
upm_df_mal_normalize.Tweet = upm_df_mal_normalize.Tweet.apply(lambda x : x['normalize'])
ukm_df_mal_normalize.Tweet = ukm_df_mal_normalize.Tweet.apply(lambda x : x['normalize'])

In [84]:
sunway_df_mal_normalize.Tweet.iloc[0]

'repostJAWATAN KOSONG DI SUNWAY UNIVERSITY21 OKTOBER 2022Silah rujuk iklan jawatan amp baca syarat kelayakan terlebih dahulu sebelum memohon melalui pautan di bawahSUNWAY UNIVERSITY'

Now the normalized strings have been extracted from dictionary.

## Spelling Correction

In [85]:
t5 = malaya.spelling_correction.transformer.transformer(model = 'tiny-t5')

INFO:malaya_boilerplate.frozen_graph:running Users\User\.cache\huggingface\hub\models--huseinzol05--spelling-correction-tiny-t5\snapshots\33b7ae5a262b35e2bd99ccedf6214b512283c438 using device /device:GPU:0


In [86]:
# Function for spelling correction
def spelling_correct(text):
    text = t5.greedy_decoder([text])
    
    return ''.join(text)

In [87]:
%%time
# Create a copy of each dataframe for spelling correction.
sunway_df_mal_splcorr = sunway_df_mal_normalize.copy()
taylor_df_mal_splcorr = taylor_df_mal_normalize.copy()
ucsi_df_mal_splcorr = ucsi_df_mal_normalize.copy()
um_df_mal_splcorr = um_df_mal_normalize.copy()
upm_df_mal_splcorr = upm_df_mal_normalize.copy()
ukm_df_mal_splcorr = ukm_df_mal_normalize.copy()

# For spelling correction.
sunway_df_mal_splcorr.Tweet = sunway_df_mal_normalize.Tweet.apply(lambda x : spelling_correct(x))
taylor_df_mal_splcorr.Tweet = taylor_df_mal_normalize.Tweet.apply(lambda x : spelling_correct(x))
ucsi_df_mal_splcorr.Tweet = ucsi_df_mal_normalize.Tweet.apply(lambda x : spelling_correct(x))
um_df_mal_splcorr.Tweet = um_df_mal_normalize.Tweet.apply(lambda x : spelling_correct(x))
upm_df_mal_splcorr.Tweet = upm_df_mal_normalize.Tweet.apply(lambda x : spelling_correct(x))
ukm_df_mal_splcorr.Tweet = ukm_df_mal_normalize.Tweet.apply(lambda x : spelling_correct(x))

CPU times: total: 28min 48s
Wall time: 1h 11min 54s


In [88]:
sunway_df_mal_splcorr.to_csv('./spelling_correction/sunway_df_mal_splcorr.csv', index=False)
taylor_df_mal_splcorr.to_csv('./spelling_correction/taylor_df_mal_splcorr.csv', index=False)
ucsi_df_mal_splcorr.to_csv('./spelling_correction/ucsi_df_mal_splcorr.csv', index=False)
um_df_mal_splcorr.to_csv('./spelling_correction/um_df_mal_splcorr.csv', index=False)
upm_df_mal_splcorr.to_csv('./spelling_correction/upm_df_mal_splcorr.csv', index=False)
ukm_df_mal_splcorr.to_csv('./spelling_correction/ukm_df_mal_splcorr.csv', index=False)

In [4]:
sunway_df_mal_splcorr = pd.read_csv('./spelling_correction/sunway_df_mal_splcorr.csv')
taylor_df_mal_splcorr = pd.read_csv('./spelling_correction/taylor_df_mal_splcorr.csv')
ucsi_df_mal_splcorr = pd.read_csv('./spelling_correction/ucsi_df_mal_splcorr.csv')
um_df_mal_splcorr = pd.read_csv('./spelling_correction/um_df_mal_splcorr.csv')
upm_df_mal_splcorr = pd.read_csv('./spelling_correction/upm_df_mal_splcorr.csv')
ukm_df_mal_splcorr = pd.read_csv('./spelling_correction/ukm_df_mal_splcorr.csv')

## Translation of Malay language and Bahasa Rojak to English

In [5]:
s_tokenizer = malaya.tokenizer.SentenceTokenizer()
transformer_noisy = malaya.translation.ms_en.transformer(model = 'noisy-base')

INFO:malaya_boilerplate.frozen_graph:running Users\User\.cache\huggingface\hub\models--huseinzol05--translation-ms-en-noisy-base\snapshots\9a661624ce95741933538f743d26235dfbee2555 using device /device:GPU:0


In [6]:
# Build a function for translation of malay languageand Bahasa Rojak to English.
def translate_malay_to_en(i):
    token_s = s_tokenizer.tokenize(i)
    # To prevent return of tokenized strings to be empty due to missing of fullstop.
    if token_s == []:
        i = i + ('.')
        token_s = s_tokenizer.tokenize(i)
    # To prevent return of tokenized strings to be empty due to misplacement of punctuation such as comma at the
    # end of strings.
    if token_s == []:
        i = re.sub(r'.,!?', '', i)
        token_s = s_tokenizer.tokenize(i)
    trans_s = transformer_noisy.greedy_decoder(token_s)
    
    return ''.join(trans_s)

In [7]:
# Create a copy of each dataframe for translation.
sunway_df_mal_translate = sunway_df_mal_splcorr.copy()
taylor_df_mal_translate = taylor_df_mal_splcorr.copy()
ucsi_df_mal_translate = ucsi_df_mal_splcorr.copy()
um_df_mal_translate = um_df_mal_splcorr.copy()
upm_df_mal_translate = upm_df_mal_splcorr.copy()
ukm_df_mal_translate = ukm_df_mal_splcorr.copy()

In [8]:
%%time
# For translation (1 dataframe per kernel due to long computation time).
sunway_df_mal_translate.Tweet = sunway_df_mal_splcorr.Tweet.apply(lambda x : translate_malay_to_en(x))

CPU times: total: 2min 39s
Wall time: 2min 24s


In [9]:
%%time
taylor_df_mal_translate.Tweet = taylor_df_mal_splcorr.Tweet.apply(lambda x : translate_malay_to_en(x))

CPU times: total: 2min 10s
Wall time: 1min 42s


In [10]:
%%time
ucsi_df_mal_translate.Tweet = ucsi_df_mal_splcorr.Tweet.apply(lambda x : translate_malay_to_en(x))

CPU times: total: 1min 18s
Wall time: 1min


In [11]:
%%time
um_df_mal_translate.Tweet = um_df_mal_splcorr.Tweet.apply(lambda x : translate_malay_to_en(x))

CPU times: total: 18min 18s
Wall time: 13min 57s


In [12]:
%%time
upm_df_mal_translate.Tweet = upm_df_mal_splcorr.Tweet.apply(lambda x : translate_malay_to_en(x))

CPU times: total: 12min 40s
Wall time: 9min 38s


In [13]:
%%time
ukm_df_mal_translate.Tweet = ukm_df_mal_splcorr.Tweet.apply(lambda x : translate_malay_to_en(x))

CPU times: total: 17min 13s
Wall time: 13min 8s


### Citation
@misc{Malaya, Speech-Toolkit library for bahasa Malaysia, powered by Deep Learning Tensorflow,
  author = {Husein, Zolkepli},
  title = {Malaya-Speech},
  year = {2020},
  publisher = {GitHub},
  journal = {GitHub repository},
  howpublished = {\url{https://github.com/huseinzol05/malaya-speech}}
}

## End of Stage 2

We then combine the tweets in English and Malay language into a dataframe for each university.

In [94]:
sunway_df_stage2 = sunway_df_stage1_en.append(sunway_df_mal_translate)
taylor_df_stage2 = taylor_df_stage1_en.append(taylor_df_mal_translate)
ucsi_df_stage2 = ucsi_df_stage1_en.append(ucsi_df_mal_translate)
um_df_stage2 = um_df_stage1_en.append(um_df_mal_translate)
upm_df_stage2 = upm_df_stage1_en.append(upm_df_mal_translate)
ukm_df_stage2 = ukm_df_stage1_en.append(ukm_df_mal_translate)

In [95]:
sunway_df_stage2.to_csv('./sunway_df_stage2.csv', index=False)
taylor_df_stage2.to_csv('./taylor_df_stage2.csv', index=False)
ucsi_df_stage2.to_csv('./ucsi_df_stage2.csv', index=False)
um_df_stage2.to_csv('./um_df_stage2.csv', index=False)
upm_df_stage2.to_csv('./upm_df_stage2.csv', index=False)
ukm_df_stage2.to_csv('./ukm_df_stage2.csv', index=False)

In [5]:
sunway_df_stage2 = pd.read_csv('./sunway_df_stage2.csv')
taylor_df_stage2 = pd.read_csv('./taylor_df_stage2.csv')
ucsi_df_stage2 = pd.read_csv('./ucsi_df_stage2.csv')
um_df_stage2 = pd.read_csv('./um_df_stage2.csv')
upm_df_stage2 = pd.read_csv('./upm_df_stage2.csv')
ukm_df_stage2 = pd.read_csv('./ukm_df_stage2.csv')

# Third Stage

## Conversion of upper case to lower case

In [6]:
# Create a copy of each dataframe for conversion of upper case to lower case.
sunway_df_lwr_to_upp = sunway_df_stage2.copy()
taylor_df_lwr_to_upp = taylor_df_stage2.copy()
ucsi_df_lwr_to_upp = ucsi_df_stage2.copy()
um_df_lwr_to_upp = um_df_stage2.copy()
upm_df_lwr_to_upp = upm_df_stage2.copy()
ukm_df_lwr_to_upp = ukm_df_stage2.copy()


# For conversion to lower case.
sunway_df_lwr_to_upp.Tweet = sunway_df_stage2.Tweet.str.lower()
taylor_df_lwr_to_upp.Tweet = taylor_df_stage2.Tweet.str.lower()
ucsi_df_lwr_to_upp.Tweet = ucsi_df_stage2.Tweet.str.lower()
um_df_lwr_to_upp.Tweet = um_df_stage2.Tweet.str.lower()
upm_df_lwr_to_upp.Tweet = upm_df_stage2.Tweet.str.lower()
ukm_df_lwr_to_upp.Tweet = ukm_df_stage2.Tweet.str.lower()

In [7]:
sunway_df_lwr_to_upp

,User,Date_Created,Number_of_Likes,Source_of_Tweet,Tweet,Language
0,jkmkhoo,2022-09-29 13:30:47+00:00,2,Twitter Web App,this restaurant at near sunway university might suit a british palette well 😉,en
1,eurospan,2022-09-29 09:46:32+00:00,0,Twitter Web App,exciting news sunway university press have launched a brandnew short story competition inspired by the un sustainable development goals. you can find out more on the eurospan blog now,en
2,SunwayCPH,2022-09-29 08:30:05+00:00,4,Twitter for iPhone,"check this out!our chief planetary health scientist, dr.is in the latest episode of the katipunan dialogue podcast 🗣️planetaryhealth sunwaycph sunwayuniversity upcids podcast",en
3,keenguyencaphe,2022-09-29 04:13:02+00:00,5,Twitter for Android,thus that was an amazing weekend as kee nguyễn went on the loose to bring vietnams finest coffee into sunway university. big kudos to keenguyensunway for all the amazing hardwork and we hope that one sip brings you all to vietnam or back to vietnam already 😉got events? dm,en
4,Mia7Jane,2022-09-28 11:16:17+00:00,1,Twitter for Android,"he asked dr jane from sunway university?, but not dr jane from malaysia... 😆 mosthappeningcampus",en
5,SunwayCPH,2022-09-27 06:52:59+00:00,3,Twitter for iPhone,truly one of the greatest🇲🇾💚 thank youfor this special feature on our executive director!planetaryhealth sunwaycph sunwayuniversity climatecrisis,en
6,SunwayCPH,2022-09-27 06:17:26+00:00,4,Twitter for iPhone,"thank youfor visiting us, exciting things ahead!unicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms",en
7,SunwayCPH,2022-09-27 06:17:23+00:00,4,Twitter for iPhone,"we gave them the tour of the wonderful sunway university campus, sunway xfarms, amp our office at the sunway university graduate centre! ​we also discussed our future joint research amp advocacy plans 🙌unicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms",en
8,SunwayCPH,2022-09-27 06:17:20+00:00,5,Twitter for iPhone,"last week, our friends fromvisited us! ​unicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms",en
9,Mia7Jane,2022-09-26 23:38:57+00:00,3,Twitter for Android,"after 2 weeks in france, a phd student stopped me and asked are you dr jane from sunway university? wah seh im popular and i know it hahaha 😆",en


## Repeated characters and consecutively repetitive words removal (spelling correction)

In [26]:
# Function for repeated characters and consecutively repetitive words removal (spelling correction)
def correct_sentence_spelling(text):
    
    # Removal of consecutively repetitive words
    correct_text = ' '.join(dict.fromkeys(text.split()))
    
    # Spelling Correction
    sentence = TextBlob(correct_text)
    correct_text = sentence.correct()
    
    return(str(correct_text))

In [27]:
# Create a copy of each dataframe for spelling correction.
sunway_df_splcor = sunway_df_lwr_to_upp.copy()
taylor_df_splcor = taylor_df_lwr_to_upp.copy()
ucsi_df_splcor = ucsi_df_lwr_to_upp.copy()
um_df_splcor = um_df_lwr_to_upp.copy()
upm_df_splcor = upm_df_lwr_to_upp.copy()
ukm_df_splcor = ukm_df_lwr_to_upp.copy()

In [28]:
%%time
# For spelling correction.
sunway_df_splcor.Tweet = sunway_df_lwr_to_upp.Tweet.apply(lambda x: correct_sentence_spelling(x))

CPU times: total: 20min 10s
Wall time: 21min 39s


In [29]:
%%time
taylor_df_splcor.Tweet = taylor_df_lwr_to_upp.Tweet.apply(lambda x: correct_sentence_spelling(x))

CPU times: total: 31min 52s
Wall time: 32min 31s


In [30]:
%%time
ucsi_df_splcor.Tweet = ucsi_df_lwr_to_upp.Tweet.apply(lambda x: correct_sentence_spelling(x))

CPU times: total: 39min 36s
Wall time: 39min 37s


In [31]:
%%time
um_df_splcor.Tweet = um_df_lwr_to_upp.Tweet.apply(lambda x: correct_sentence_spelling(x))

CPU times: total: 1h 14min
Wall time: 1h 14min 2s


In [32]:
%%time
upm_df_splcor.Tweet = upm_df_lwr_to_upp.Tweet.apply(lambda x: correct_sentence_spelling(x))

CPU times: total: 34min 28s
Wall time: 34min 30s


In [33]:
%%time
ukm_df_splcor.Tweet = ukm_df_lwr_to_upp.Tweet.apply(lambda x: correct_sentence_spelling(x))

CPU times: total: 44min 58s
Wall time: 44min 59s


In [34]:
sunway_df_splcor.to_csv('./spelling_correction/sunway_df_splcor.csv', index=False)
taylor_df_splcor.to_csv('./spelling_correction/taylor_df_splcor.csv', index=False)
ucsi_df_splcor.to_csv('./spelling_correction/ucsi_df_splcor.csv', index=False)
um_df_splcor.to_csv('./spelling_correction/um_df_splcor.csv', index=False)
upm_df_splcor.to_csv('./spelling_correction/upm_df_splcor.csv', index=False)
ukm_df_splcor.to_csv('./spelling_correction/ukm_df_splcor.csv', index=False)

In [35]:
sunway_df_splcor = pd.read_csv('./spelling_correction/sunway_df_splcor.csv')
taylor_df_splcor = pd.read_csv('./spelling_correction/taylor_df_splcor.csv')
ucsi_df_splcor = pd.read_csv('./spelling_correction/ucsi_df_splcor.csv')
um_df_splcor = pd.read_csv('./spelling_correction/um_df_splcor.csv')
upm_df_splcor = pd.read_csv('./spelling_correction/upm_df_splcor.csv')
ukm_df_splcor = pd.read_csv('./spelling_correction/ukm_df_splcor.csv')

## Informal to formal word conversion

In [36]:
# Create a copy of each dataframe for converting informal word to formal word.
sunway_df_inftofor = sunway_df_splcor.copy()
taylor_df_inftofor = taylor_df_splcor.copy()
ucsi_df_inftofor = ucsi_df_splcor.copy()
um_df_inftofor = um_df_splcor.copy()
upm_df_inftofor = upm_df_splcor.copy()
ukm_df_inftofor = ukm_df_splcor.copy()

# Convert 'u' to 'you'.
sunway_df_inftofor.Tweet = sunway_df_splcor.Tweet.str.replace('\su\s', 'you', regex=True)
taylor_df_inftofor.Tweet = taylor_df_splcor.Tweet.str.replace('\su\s', 'you', regex=True)
ucsi_df_inftofor.Tweet = ucsi_df_splcor.Tweet.str.replace('\su\s', 'you', regex=True)
um_df_inftofor.Tweet = um_df_splcor.Tweet.str.replace('\su\s', 'you', regex=True)
upm_df_inftofor.Tweet = upm_df_splcor.Tweet.str.replace('\su\s', 'you', regex=True)
ukm_df_inftofor.Tweet = ukm_df_splcor.Tweet.str.replace('\su\s', 'you', regex=True)

# Convert 'tq' to thank you.
sunway_df_inftofor.Tweet = sunway_df_inftofor.Tweet.str.replace('\stq\s', 'thank you', regex=True)
taylor_df_inftofor.Tweet = taylor_df_inftofor.Tweet.str.replace('\stq\s', 'thank you', regex=True)
ucsi_df_inftofor.Tweet = ucsi_df_inftofor.Tweet.str.replace('\stq\s', 'thank you', regex=True)
um_df_inftofor.Tweet = um_df_inftofor.Tweet.str.replace('\stq\s', 'thank you', regex=True)
upm_df_inftofor.Tweet = upm_df_inftofor.Tweet.str.replace('\stq\s', 'thank you', regex=True)
ukm_df_inftofor.Tweet = ukm_df_inftofor.Tweet.str.replace('\stq\s', 'thank you', regex=True)

# Convert 'uni' to 'university'.
sunway_df_inftofor.Tweet = sunway_df_inftofor.Tweet.str.replace('\s?uni\s', 'university', regex=True)
taylor_df_inftofor.Tweet = taylor_df_inftofor.Tweet.str.replace('\s?uni\s', 'university', regex=True)
ucsi_df_inftofor.Tweet = ucsi_df_inftofor.Tweet.str.replace('\s?uni\s', 'university', regex=True)
um_df_inftofor.Tweet = um_df_inftofor.Tweet.str.replace('\s?uni\s', 'university', regex=True)
upm_df_inftofor.Tweet = upm_df_inftofor.Tweet.str.replace('\s?uni\s', 'university', regex=True)
ukm_df_inftofor.Tweet = ukm_df_inftofor.Tweet.str.replace('\s?uni\s', 'university', regex=True)

# Convert 'nt' to 'not'.
sunway_df_inftofor.Tweet = sunway_df_inftofor.Tweet.str.replace('\snt\s', 'not', regex=True)
taylor_df_inftofor.Tweet = taylor_df_inftofor.Tweet.str.replace('\snt\s', 'not', regex=True)
ucsi_df_inftofor.Tweet = ucsi_df_inftofor.Tweet.str.replace('\snt\s', 'not', regex=True)
um_df_inftofor.Tweet = um_df_inftofor.Tweet.str.replace('\snt\s', 'not', regex=True)
upm_df_inftofor.Tweet = upm_df_inftofor.Tweet.str.replace('\snt\s', 'not', regex=True)
ukm_df_inftofor.Tweet = ukm_df_inftofor.Tweet.str.replace('\snt\s', 'not', regex=True)

# Convert 'malasha' to 'malaysia'
sunway_df_inftofor.Tweet = sunway_df_inftofor.Tweet.str.replace('malasha', 'malaysia', regex=True)
taylor_df_inftofor.Tweet = taylor_df_inftofor.Tweet.str.replace('malasha', 'malaysia', regex=True)
ucsi_df_inftofor.Tweet = ucsi_df_inftofor.Tweet.str.replace('malasha', 'malaysia', regex=True)
um_df_inftofor.Tweet = um_df_inftofor.Tweet.str.replace('malasha', 'malaysia', regex=True)
upm_df_inftofor.Tweet = upm_df_inftofor.Tweet.str.replace('malasha', 'malaysia', regex=True)
ukm_df_inftofor.Tweet = ukm_df_inftofor.Tweet.str.replace('malasha', 'malaysia', regex=True)

## Contractions expansion

In [37]:
contractions = {
    "ain't": "am not / are not", "aren't": "are not / am not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", 
    "could've": "could have", "couldn't": "could not", "couldn't've": "could not have", "didn't": "did not", 
    "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have", "hasn't": "has not", 
    "haven't": "have not", "he'd": "he had / he would", "he'd've": "he would have","he'll": "he shall / he will", 
    "he'll've": "he shall have / he will have", "he's": "he has / he is", "how'd": "how did", "how'd'y": "how do you", 
    "how'll": "how will", "how's": "how has / how is", "i'd": "I had / I would", "i'd've": "I would have", 
    "i'll": "i shall / i will", "i'll've": "i shall have / i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", 
    "it'd": "it had / it would", "it'd've": "it would have", "it'll": "it shall / it will", 
    "it'll've": "it shall have / it will have", "it's": "it has / it is", "let's": "let us", "ma'am": "madam", 
    "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", 
    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
    "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", 
    "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she had / she would", 
    "she'd've": "she would have", "she'll": "she shall / she will", "she'll've": "she shall have / she will have", 
    "she's": "she has / she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", 
    "so've": "so have", "so's": "so as / so is", "that'd": "that would / that had", "that'd've": "that would have", 
    "that's": "that has / that is", "there'd": "there had / there would", "there'd've": "there would have", 
    "there's": "there has / there is", "they'd": "they had / they would", "they'd've": "they would have",
    "they'll": "they shall / they will", "they'll've": "they shall have / they will have", "they're": "they are", 
    "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we had / we would", "we'd've": "we would have",
    "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what shall / what will", 
    "what'll've": "what shall have / what will have", "what're": "what are", "what's": "what has / what is", 
    "what've": "what have", "when's": "when has / when is", "when've": "when have", "where'd": "where did", 
    "where's": "where has / where is", "where've": "where have", "who'll": "who shall / who will", 
    "who'll've": "who shall have / who will have", "who's": "who has / who is", "who've": "who have", 
    "why's": "why has / why is", "why've": "why have", "will've": "will have", "won't": "will not", 
    "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
    "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have", "y'all're": "you all are", 
    "y'all've": "you all have", "you'd": "you had / you would", "you'd've": "you would have", "you'll": "you shall / you will", 
    "you'll've": "you shall have / you will have", "you're": "you are", "you've": "you have"
}

In [38]:
def contraction_expansion(text):
    for word in text.split():
        if word in contractions:
            text = text.replace(word, contractions[word])
    
    return text

In [39]:
# Create a copy of each dataframe for contraction expansion.
sunway_df_conexp = sunway_df_inftofor.copy()
taylor_df_conexp = taylor_df_inftofor.copy()
ucsi_df_conexp = ucsi_df_inftofor.copy()
um_df_conexp = um_df_inftofor.copy()
upm_df_conexp = upm_df_inftofor.copy()
ukm_df_conexp = ukm_df_inftofor.copy()

# For contraction expansion.
sunway_df_conexp.Tweet = sunway_df_inftofor.Tweet.apply(lambda x :contraction_expansion(x))
taylor_df_conexp.Tweet = taylor_df_inftofor.Tweet.apply(lambda x :contraction_expansion(x))
ucsi_df_conexp.Tweet = ucsi_df_inftofor.Tweet.apply(lambda x :contraction_expansion(x))
um_df_conexp.Tweet = um_df_inftofor.Tweet.apply(lambda x :contraction_expansion(x))
upm_df_conexp.Tweet = upm_df_inftofor.Tweet.apply(lambda x :contraction_expansion(x))
ukm_df_conexp.Tweet = ukm_df_inftofor.Tweet.apply(lambda x :contraction_expansion(x))

## Numeric characters removal

In [40]:
# Create a copy of each dataframe for numeric characters removal.
sunway_df_num_rm = sunway_df_conexp.copy()
taylor_df_num_rm = taylor_df_conexp.copy()
ucsi_df_num_rm = ucsi_df_conexp.copy()
um_df_num_rm = um_df_conexp.copy()
upm_df_num_rm = upm_df_conexp.copy()
ukm_df_num_rm = ukm_df_conexp.copy()

# For numeric characters removal.
sunway_df_num_rm.Tweet = sunway_df_conexp.Tweet.str.replace('\d+','', regex=True)
taylor_df_num_rm.Tweet = taylor_df_conexp.Tweet.str.replace('\d+','', regex=True)
ucsi_df_num_rm.Tweet = ucsi_df_conexp.Tweet.str.replace('\d+','', regex=True)
um_df_num_rm.Tweet = um_df_conexp.Tweet.str.replace('\d+','', regex=True)
upm_df_num_rm.Tweet = upm_df_conexp.Tweet.str.replace('\d+','', regex=True)
ukm_df_num_rm.Tweet = ukm_df_conexp.Tweet.str.replace('\d+','', regex=True)

## Emoji-text conversion 

In [41]:
# Function for emoji-text conversion 
def convert_emojis(text):
    extracted_text = emoji.demojize(text, delimiters=('', ''))
    return extracted_text

In [42]:
# Create a copy of each dataframe for emoji-text conversion.
sunway_df_emj_cnv = sunway_df_num_rm.copy()
taylor_df_emj_cnv = taylor_df_num_rm.copy()
ucsi_df_emj_cnv = ucsi_df_num_rm.copy()
um_df_emj_cnv = um_df_num_rm.copy()
upm_df_emj_cnv = upm_df_num_rm.copy()
ukm_df_emj_cnv = ukm_df_num_rm.copy()

# To convert emoji to text.
sunway_df_emj_cnv.Tweet = sunway_df_num_rm.Tweet.apply(lambda x: convert_emojis(x))
taylor_df_emj_cnv.Tweet = taylor_df_num_rm.Tweet.apply(lambda x: convert_emojis(x))
ucsi_df_emj_cnv.Tweet = ucsi_df_num_rm.Tweet.apply(lambda x: convert_emojis(x))
um_df_emj_cnv.Tweet = um_df_num_rm.Tweet.apply(lambda x: convert_emojis(x))
upm_df_emj_cnv.Tweet = upm_df_num_rm.Tweet.apply(lambda x: convert_emojis(x))
ukm_df_emj_cnv.Tweet = ukm_df_num_rm.Tweet.apply(lambda x: convert_emojis(x))

In [43]:
sunway_df_emj_cnv[['Tweet']]

,Tweet
0,this restaurant at near sunday university might suit a british palate well winking_face
1,exciting news sunday university press have launched a branded short story competition inspired by the un sustainable development goals. you can find out more on european blow now
2,"check this out!our chief plantar health scientist, dr.is in the latest episode of katipunan dialogue podcast speaking_headplanetaryhealth sunwaycph sunwayuniversity upside"
3,thus that was an amazing weekend as see nguyễn went on the loose to bring vietnams finest coffee into sunday university. big buds keenguyensunway for all hardware and we hope one sip brings you vienna or back already winking_facegot events? do
4,"he asked dr jane from sunday university?, but not malaysia... grinning_squinting_face mosthappeningcampus"
5,truly one of the greatestMalaysiagreen_heart thank your this special feature on our executive director!planetaryhealth sunwaycph sunwayuniversity climatecrisis
6,"thank your visiting us, exciting things ahead!unicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms"
7,"we gave them the tour of wonderful sunday university camps, farms, amp our office at graduate centre! ​we also discussed future joint research advocacy plans raising_handsunicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms"
8,"last week, our friends fromvisited us! ​unicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms"
9,"after weeks in france, a pad student stopped me and asked are you dr jane from sunday university? was she in popular i know it havana grinning_squinting_face"


## Punctuation (all) and extra whitespaces removal

This step involves removing all punctuations from the text.. The Python string library has a predefined collection of punctuation characters such as ‘!”#$%&'()*+,-./:;?@[]^_`{|}~’

In [44]:
# Function for punctuation removal.
def remove_punctuation_xwhitespaces(text):
    text = ''.join(char for char in text if char not in string.punctuation)
    text = re.sub('\s\s+','', text)
    return text

In [45]:
# Create a copy of each dataframe for punctuation removal.
sunway_df_punc_rm = sunway_df_emj_cnv.copy()
taylor_df_punc_rm = taylor_df_emj_cnv.copy()
ucsi_df_punc_rm = ucsi_df_emj_cnv.copy()
um_df_punc_rm = um_df_emj_cnv.copy()
upm_df_punc_rm = upm_df_emj_cnv.copy()
ukm_df_punc_rm = ukm_df_emj_cnv.copy()

# To remove punctuation and extra whitespaces.
sunway_df_punc_rm.Tweet = sunway_df_emj_cnv.Tweet.apply(lambda x: remove_punctuation_xwhitespaces(x))
taylor_df_punc_rm.Tweet = taylor_df_emj_cnv.Tweet.apply(lambda x: remove_punctuation_xwhitespaces(x))
ucsi_df_punc_rm.Tweet= ucsi_df_emj_cnv.Tweet.apply(lambda x: remove_punctuation_xwhitespaces(x))
um_df_punc_rm.Tweet = um_df_emj_cnv.Tweet.apply(lambda x: remove_punctuation_xwhitespaces(x))
upm_df_punc_rm.Tweet = upm_df_emj_cnv.Tweet.apply(lambda x: remove_punctuation_xwhitespaces(x))
ukm_df_punc_rm.Tweet = ukm_df_emj_cnv.Tweet.apply(lambda x: remove_punctuation_xwhitespaces(x))

In [46]:
sunway_df_punc_rm[['Tweet']]

,Tweet
0,this restaurant at near sunday university might suit a british palate well winkingface
1,exciting news sunday university press have launched a branded short story competition inspired by the un sustainable development goals you can find out more on european blow now
2,check this outour chief plantar health scientist dris in the latest episode of katipunan dialogue podcast speakingheadplanetaryhealth sunwaycph sunwayuniversity upside
3,thus that was an amazing weekend as see nguyễn went on the loose to bring vietnams finest coffee into sunday university big buds keenguyensunway for all hardware and we hope one sip brings you vienna or back already winkingfacegot events do
4,he asked dr jane from sunday university but not malaysia grinningsquintingface mosthappeningcampus
5,truly one of the greatestMalaysiagreenheart thank your this special feature on our executive directorplanetaryhealth sunwaycph sunwayuniversity climatecrisis
6,thank your visiting us exciting things aheadunicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms
7,we gave them the tour of wonderful sunday university camps farms amp our office at graduate centre ​we also discussed future joint research advocacy plans raisinghandsunicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms
8,last week our friends fromvisited us ​unicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms
9,afterweeks in france a pad student stopped me and asked are you dr jane from sunday university was she in popular i know it havana grinningsquintingface


## End of Stage 3

# Fourth stage

## Tokenization

In [47]:
# Function for tokenization
def tokenization(text):
    ton_text = re.split('\W+', text)
    
    return ton_text

In [48]:
# Create a copy of each dataframe for tokenization.
sunway_df_token = sunway_df_punc_rm.copy()
taylor_df_token = taylor_df_punc_rm.copy()
ucsi_df_token = ucsi_df_punc_rm.copy()
um_df_token = um_df_punc_rm.copy()
upm_df_token = upm_df_punc_rm.copy()
ukm_df_token = ukm_df_punc_rm.copy()

# For tokenization.
sunway_df_token['Tokenized_Tweet'] = sunway_df_punc_rm.Tweet.apply(lambda x: tokenization(x))
taylor_df_token['Tokenized_Tweet'] = taylor_df_punc_rm.Tweet.apply(lambda x: tokenization(x))
ucsi_df_token['Tokenized_Tweet'] = ucsi_df_punc_rm.Tweet.apply(lambda x: tokenization(x))
um_df_token['Tokenized_Tweet'] = um_df_punc_rm.Tweet.apply(lambda x: tokenization(x))
upm_df_token['Tokenized_Tweet'] = upm_df_punc_rm.Tweet.apply(lambda x: tokenization(x))
ukm_df_token['Tokenized_Tweet'] = ukm_df_punc_rm.Tweet.apply(lambda x: tokenization(x))

In [49]:
sunway_df_token[['Tokenized_Tweet']]

,Tokenized_Tweet
0,"[this, restaurant, at, near, sunday, university, might, suit, a, british, palate, well, winkingface]"
1,"[exciting, news, sunday, university, press, have, launched, a, branded, short, story, competition, inspired, by, the, un, sustainable, development, goals, you, can, find, out, more, on, european, blow, now]"
2,"[check, this, outour, chief, plantar, health, scientist, dris, in, the, latest, episode, of, katipunan, dialogue, podcast, speakingheadplanetaryhealth, sunwaycph, sunwayuniversity, upside]"
3,"[thus, that, was, an, amazing, weekend, as, see, nguyễn, went, on, the, loose, to, bring, vietnams, finest, coffee, into, sunday, university, big, buds, keenguyensunway, for, all, hardware, and, we, hope, one, sip, brings, you, vienna, or, back, already, winkingfacegot, events, do]"
4,"[he, asked, dr, jane, from, sunday, university, but, not, malaysia, grinningsquintingface, mosthappeningcampus]"
5,"[truly, one, of, the, greatestMalaysiagreenheart, thank, your, this, special, feature, on, our, executive, directorplanetaryhealth, sunwaycph, sunwayuniversity, climatecrisis]"
6,"[thank, your, visiting, us, exciting, things, aheadunicefmalaysia, sunwaycph, planetaryhealth, sunwayuniversity, sunwayxfarms]"
7,"[we, gave, them, the, tour, of, wonderful, sunday, university, camps, farms, amp, our, office, at, graduate, centre, we, also, discussed, future, joint, research, advocacy, plans, raisinghandsunicefmalaysia, sunwaycph, planetaryhealth, sunwayuniversity, sunwayxfarms]"
8,"[last, week, our, friends, fromvisited, us, unicefmalaysia, sunwaycph, planetaryhealth, sunwayuniversity, sunwayxfarms]"
9,"[afterweeks, in, france, a, pad, student, stopped, me, and, asked, are, you, dr, jane, from, sunday, university, was, she, in, popular, i, know, it, havana, grinningsquintingface]"


## Stop words removal

In [50]:
stopword = nltk.corpus.stopwords.words('english')

In [51]:
# Function for stopword removal.
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

In [52]:
# Create a copy of each dataframe for stop words removal.
sunway_df_stpwrd = sunway_df_token.copy()
taylor_df_stpwrd = taylor_df_token.copy()
ucsi_df_stpwrd = ucsi_df_token.copy()
um_df_stpwrd = um_df_token.copy()
upm_df_stpwrd = upm_df_token.copy()
ukm_df_stpwrd = ukm_df_token.copy()

# For stopwords removal.
sunway_df_stpwrd.Tokenized_Tweet = sunway_df_token.Tokenized_Tweet.apply(lambda x: remove_stopwords(x))
taylor_df_stpwrd.Tokenized_Tweet = taylor_df_token.Tokenized_Tweet.apply(lambda x: remove_stopwords(x))
ucsi_df_stpwrd.Tokenized_Tweet = ucsi_df_token.Tokenized_Tweet.apply(lambda x: remove_stopwords(x))
um_df_stpwrd.Tokenized_Tweet = um_df_token.Tokenized_Tweet.apply(lambda x: remove_stopwords(x))
upm_df_stpwrd.Tokenized_Tweet = upm_df_token.Tokenized_Tweet.apply(lambda x: remove_stopwords(x))
ukm_df_stpwrd.Tokenized_Tweet = ukm_df_token.Tokenized_Tweet.apply(lambda x: remove_stopwords(x))

In [53]:
sunway_df_stpwrd[['Tokenized_Tweet']]

,Tokenized_Tweet
0,"[restaurant, near, sunday, university, might, suit, british, palate, well, winkingface]"
1,"[exciting, news, sunday, university, press, launched, branded, short, story, competition, inspired, un, sustainable, development, goals, find, european, blow]"
2,"[check, outour, chief, plantar, health, scientist, dris, latest, episode, katipunan, dialogue, podcast, speakingheadplanetaryhealth, sunwaycph, sunwayuniversity, upside]"
3,"[thus, amazing, weekend, see, nguyễn, went, loose, bring, vietnams, finest, coffee, sunday, university, big, buds, keenguyensunway, hardware, hope, one, sip, brings, vienna, back, already, winkingfacegot, events]"
4,"[asked, dr, jane, sunday, university, malaysia, grinningsquintingface, mosthappeningcampus]"
5,"[truly, one, greatestMalaysiagreenheart, thank, special, feature, executive, directorplanetaryhealth, sunwaycph, sunwayuniversity, climatecrisis]"
6,"[thank, visiting, us, exciting, things, aheadunicefmalaysia, sunwaycph, planetaryhealth, sunwayuniversity, sunwayxfarms]"
7,"[gave, tour, wonderful, sunday, university, camps, farms, amp, office, graduate, centre, also, discussed, future, joint, research, advocacy, plans, raisinghandsunicefmalaysia, sunwaycph, planetaryhealth, sunwayuniversity, sunwayxfarms]"
8,"[last, week, friends, fromvisited, us, unicefmalaysia, sunwaycph, planetaryhealth, sunwayuniversity, sunwayxfarms]"
9,"[afterweeks, france, pad, student, stopped, asked, dr, jane, sunday, university, popular, know, havana, grinningsquintingface]"


## Lemmatization

In [54]:
wordnet_lemmatizer = WordNetLemmatizer()

In [55]:
# Function for lemmatization.
def lemmatize(text):
    text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return ' '.join(text)

In [56]:
# Create a copy of each dataframe for lemmatization.
sunway_df_lemm = sunway_df_stpwrd.copy()
taylor_df_lemm = taylor_df_stpwrd.copy()
ucsi_df_lemm = ucsi_df_stpwrd.copy()
um_df_lemm = um_df_stpwrd.copy()
upm_df_lemm = upm_df_stpwrd.copy()
ukm_df_lemm = ukm_df_stpwrd.copy()

# For lemmatization.
sunway_df_lemm.Tokenized_Tweet = sunway_df_stpwrd.Tokenized_Tweet.apply(lambda x : lemmatize(x))
taylor_df_lemm.Tokenized_Tweet = taylor_df_stpwrd.Tokenized_Tweet.apply(lambda x : lemmatize(x))
ucsi_df_lemm.Tokenized_Tweet = ucsi_df_stpwrd.Tokenized_Tweet.apply(lambda x : lemmatize(x))
um_df_lemm.Tokenized_Tweet = um_df_stpwrd.Tokenized_Tweet.apply(lambda x : lemmatize(x))
upm_df_lemm.Tokenized_Tweet = upm_df_stpwrd.Tokenized_Tweet.apply(lambda x : lemmatize(x))
ukm_df_lemm.Tokenized_Tweet = ukm_df_stpwrd.Tokenized_Tweet.apply(lambda x : lemmatize(x))

In [57]:
sunway_df_lemm[['Tokenized_Tweet']]

,Tokenized_Tweet
0,restaurant near sunday university might suit british palate well winkingface
1,exciting news sunday university press launched branded short story competition inspired un sustainable development goal find european blow
2,check outour chief plantar health scientist dris latest episode katipunan dialogue podcast speakingheadplanetaryhealth sunwaycph sunwayuniversity upside
3,thus amazing weekend see nguyễn went loose bring vietnam finest coffee sunday university big bud keenguyensunway hardware hope one sip brings vienna back already winkingfacegot event
4,asked dr jane sunday university malaysia grinningsquintingface mosthappeningcampus
5,truly one greatestMalaysiagreenheart thank special feature executive directorplanetaryhealth sunwaycph sunwayuniversity climatecrisis
6,thank visiting u exciting thing aheadunicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms
7,gave tour wonderful sunday university camp farm amp office graduate centre also discussed future joint research advocacy plan raisinghandsunicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms
8,last week friend fromvisited u unicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms
9,afterweeks france pad student stopped asked dr jane sunday university popular know havana grinningsquintingface


## End of Stage 4

Since it is the end of fourth stage, we assign the name 'stage_3' to all dataframes.

In [58]:
sunway_df_stage4 = sunway_df_lemm.copy()
taylor_df_stage4 = taylor_df_lemm.copy()
ucsi_df_stage4 = ucsi_df_lemm.copy()
um_df_stage4 = um_df_lemm.copy()
upm_df_stage4 = upm_df_lemm.copy()
ukm_df_stage4 = ukm_df_lemm.copy()

In [59]:
sunway_df_stage4.to_csv('./sunway_df_stage4.csv', index=False)
taylor_df_stage4.to_csv('./taylor_df_stage4.csv', index=False)
ucsi_df_stage4.to_csv('./ucsi_df_stage4.csv', index=False)
um_df_stage4.to_csv('./um_df_stage4.csv', index=False)
upm_df_stage4.to_csv('./upm_df_stage4.csv', index=False)
ukm_df_stage4.to_csv('./ukm_df_stage4.csv', index=False)

In [60]:
sunway_df_stage4 = pd.read_csv('./sunway_df_stage4.csv')
taylor_df_stage4 = pd.read_csv('./taylor_df_stage4.csv')
ucsi_df_stage4 = pd.read_csv('./ucsi_df_stage4.csv')
um_df_stage4 = pd.read_csv('./um_df_stage4.csv')
upm_df_stage4 = pd.read_csv('./upm_df_stage4.csv')
ukm_df_stage4 = pd.read_csv('./ukm_df_stage4.csv')

# Feature Extraction

In [61]:
# # Calculating tweet’s length and word counts
# sunway_df_stage4['Tweet_Length'] = sunway_df_stage4['Tweet'].astype(str).apply(len)
# sunway_df_stage4['Tweet_Word_Count'] = sunway_df_stage4['Tweet'].apply(lambda x: len(str(x).split()))
# taylor_df_stage4['Tweet_Length'] = taylor_df_stage4['Tweet'].astype(str).apply(len)
# taylor_df_stage4['Tweet_Word_Count'] = taylor_df_stage4['Tweet'].apply(lambda x: len(str(x).split()))
# ucsi_df_stage4['Tweet_Length'] = ucsi_df_stage4['Tweet'].astype(str).apply(len)
# ucsi_df_stage4['Tweet_Word_Count'] = ucsi_df_stage4['Tweet'].apply(lambda x: len(str(x).split()))
# um_df_stage4['Tweet_Length'] = um_df_stage4['Tweet'].astype(str).apply(len)
# um_df_stage4['Tweet_Word_Count'] = um_df_stage4['Tweet'].apply(lambda x: len(str(x).split()))
# upm_df_stage4['Tweet_Length'] = upm_df_stage4['Tweet'].astype(str).apply(len)
# upm_df_stage4['Tweet_Word_Count'] = upm_df_stage4['Tweet'].apply(lambda x: len(str(x).split()))
# ukm_df_stage4['Tweet_Length'] = ukm_df_stage4['Tweet'].astype(str).apply(len)
# ukm_df_stage4['Tweet_Word_Count'] = ukm_df_stage4['Tweet'].apply(lambda x: len(str(x).split()))

In [62]:
# # Extract Polarity and Subjectivity of tweets using TextBlob.
# sunway_df_stage4[['Polarity', 'Subjectivity']] = sunway_df_stage4.Tweet.apply(lambda x: pd.Series(TextBlob(x).sentiment))
# taylor_df_stage4[['Polarity', 'Subjectivity']] = taylor_df_stage4.Tweet.apply(lambda x: pd.Series(TextBlob(x).sentiment))
# ucsi_df_stage4[['Polarity', 'Subjectivity']] = ucsi_df_stage4.Tweet.apply(lambda x: pd.Series(TextBlob(x).sentiment))
# um_df_stage4[['Polarity', 'Subjectivity']] = um_df_stage4.Tweet.apply(lambda x: pd.Series(TextBlob(x).sentiment))
# upm_df_stage4[['Polarity', 'Subjectivity']] = upm_df_stage4.Tweet.apply(lambda x: pd.Series(TextBlob(x).sentiment))
# ukm_df_stage4[['Polarity', 'Subjectivity']] = ukm_df_stage4.Tweet.apply(lambda x: pd.Series(TextBlob(x).sentiment))

In [63]:
# Function to calculate Negative, Positive, Neutral and Compound values
def feature_extraction(text):
    score = SentimentIntensityAnalyzer().polarity_scores(text)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    sentiment = []
    if neg > pos:
        sentiment = 'negative'
    elif pos > neg:
        sentiment = 'positive'
    else:
        sentiment = 'neutral'
    
    return neg, neu, pos, comp, sentiment

In [64]:
%%time
# For calculation of negative, positive, neutral, and compound values and generation of sentiment values. 
sunway_df_stage4[['Negative', 'Neutral', 'Positive', 'Compound', 'Sentiment']] = sunway_df_stage4.Tweet.apply(
    lambda x: pd.Series(feature_extraction(x)))
taylor_df_stage4[['Negative', 'Neutral', 'Positive', 'Compound', 'Sentiment']] = taylor_df_stage4.Tweet.apply(
    lambda x: pd.Series(feature_extraction(x)))
ucsi_df_stage4[['Negative', 'Neutral', 'Positive', 'Compound', 'Sentiment']] = ucsi_df_stage4.Tweet.apply(
    lambda x: pd.Series(feature_extraction(x)))
um_df_stage4[['Negative', 'Neutral', 'Positive', 'Compound', 'Sentiment']] = um_df_stage4.Tweet.apply(
    lambda x: pd.Series(feature_extraction(x)))
upm_df_stage4[['Negative', 'Neutral', 'Positive', 'Compound', 'Sentiment']] = upm_df_stage4.Tweet.apply(
    lambda x: pd.Series(feature_extraction(x)))
ukm_df_stage4[['Negative', 'Neutral', 'Positive', 'Compound', 'Sentiment']] = ukm_df_stage4.Tweet.apply(
    lambda x: pd.Series(feature_extraction(x)))

CPU times: total: 57.2 s
Wall time: 57.3 s


In [65]:
sunway_df_stage4

,User,Date_Created,Number_of_Likes,Source_of_Tweet,Tweet,Language,Tokenized_Tweet,Negative,Neutral,Positive,Compound,Sentiment
0,jkmkhoo,2022-09-29 13:30:47+00:00,2,Twitter Web App,this restaurant at near sunday university might suit a british palate well winkingface,en,restaurant near sunday university might suit british palate well winkingface,0.000,0.840,0.160,0.2732,positive
1,eurospan,2022-09-29 09:46:32+00:00,0,Twitter Web App,exciting news sunday university press have launched a branded short story competition inspired by the un sustainable development goals you can find out more on european blow now,en,exciting news sunday university press launched branded short story competition inspired un sustainable development goal find european blow,0.000,0.752,0.248,0.7845,positive
2,SunwayCPH,2022-09-29 08:30:05+00:00,4,Twitter for iPhone,check this outour chief plantar health scientist dris in the latest episode of katipunan dialogue podcast speakingheadplanetaryhealth sunwaycph sunwayuniversity upside,en,check outour chief plantar health scientist dris latest episode katipunan dialogue podcast speakingheadplanetaryhealth sunwaycph sunwayuniversity upside,0.000,1.000,0.000,0.0000,neutral
3,keenguyencaphe,2022-09-29 04:13:02+00:00,5,Twitter for Android,thus that was an amazing weekend as see nguyễn went on the loose to bring vietnams finest coffee into sunday university big buds keenguyensunway for all hardware and we hope one sip brings you vienna or back already winkingfacegot events do,en,thus amazing weekend see nguyễn went loose bring vietnam finest coffee sunday university big bud keenguyensunway hardware hope one sip brings vienna back already winkingfacegot event,0.049,0.809,0.143,0.6597,positive
4,Mia7Jane,2022-09-28 11:16:17+00:00,1,Twitter for Android,he asked dr jane from sunday university but not malaysia grinningsquintingface mosthappeningcampus,en,asked dr jane sunday university malaysia grinningsquintingface mosthappeningcampus,0.000,1.000,0.000,0.0000,neutral
5,SunwayCPH,2022-09-27 06:52:59+00:00,3,Twitter for iPhone,truly one of the greatestMalaysiagreenheart thank your this special feature on our executive directorplanetaryhealth sunwaycph sunwayuniversity climatecrisis,en,truly one greatestMalaysiagreenheart thank special feature executive directorplanetaryhealth sunwaycph sunwayuniversity climatecrisis,0.000,0.633,0.367,0.7964,positive
6,SunwayCPH,2022-09-27 06:17:26+00:00,4,Twitter for iPhone,thank your visiting us exciting things aheadunicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms,en,thank visiting u exciting thing aheadunicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms,0.000,0.612,0.388,0.6908,positive
7,SunwayCPH,2022-09-27 06:17:23+00:00,4,Twitter for iPhone,we gave them the tour of wonderful sunday university camps farms amp our office at graduate centre ​we also discussed future joint research advocacy plans raisinghandsunicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms,en,gave tour wonderful sunday university camp farm amp office graduate centre also discussed future joint research advocacy plan raisinghandsunicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms,0.000,0.887,0.113,0.5719,positive
8,SunwayCPH,2022-09-27 06:17:20+00:00,5,Twitter for iPhone,last week our friends fromvisited us ​unicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms,en,last week friend fromvisited u unicefmalaysia sunwaycph planetaryhealth sunwayuniversity sunwayxfarms,0.000,0.763,0.237,0.4767,positive
9,Mia7Jane,2022-09-26 23:38:57+00:00,3,Twitter for Android,afterweeks in france a pad student stopped me and asked are you dr jane from sunday university was she in popular i know it havana grinningsquintingface,en,afterweeks france pad student stopped asked dr jane sunday university popular know havana grinningsquintingface,0.071,0.824,0.105,0.2263,positive


In [ ]:
sunway_df_stage4.to_csv('sunway_df_processed.csv', index=False)
taylor_df_stage4.to_csv('taylor_df_processed.csv', index=False)
ucsi_df_stage4.to_csv('ucsi_df_processed.csv', index=False)
um_df_stage4.to_csv('um_df_processed.csv', index=False)
upm_df_stage4.to_csv('upm_df_processed.csv', index=False)
ukm_df_stage4.to_csv('ukm_df_processed.csv', index=False)